In [1]:
import os

In [2]:
from crawler import crawler

In [3]:
number_of_dog_pages = 151
dog_url = 'https://www.freeimages.com/search/dog/'
dog_download_path = 'data/dog/'

In [4]:
if not os.path.exists(dog_download_path):
    os.makedirs(dog_download_path)

In [5]:
for i in range(1, number_of_dog_pages + 1):
    crawler(dog_url, i, dog_download_path)

In [6]:
number_of_cat_pages = 88
cat_url = 'https://www.freeimages.com/search/cat/'
cat_download_path = 'data/cat/'

In [7]:
if not os.path.exists(cat_download_path):
    os.makedirs(cat_download_path)

In [8]:
for i in range(1, number_of_cat_pages + 1):
    crawler(cat_url, i, cat_download_path)

In [9]:
data_path = '/content/data'

In [10]:
image_size = (224, 224)
batch_size = 32

In [11]:
labels = ["CAT", "DOG"]

In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.11.0'

In [14]:
dog_data_path = '/content/data/dog'
cat_data_path = '/content/data/cat'

In [15]:
from tensorflow.keras.utils import image_dataset_from_directory

In [16]:
dataset = image_dataset_from_directory(data_path, batch_size= batch_size, image_size= image_size, shuffle= True)

Found 7858 files belonging to 2 classes.


In [17]:
len(dataset)

246

In [18]:
import math

In [19]:
train = dataset.take(math.ceil(len(dataset)*.75))
test = dataset.skip(math.ceil(len(dataset)*.75)).take(math.floor(len(dataset)*.25))

In [20]:
samples, labels = train.take(1).as_numpy_iterator().next()
samples.shape

(32, 224, 224, 3)

In [21]:
input_shape = samples.shape[1:]
input_shape

(224, 224, 3)

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Rescaling, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [23]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.resnet import ResNet101

In [24]:
epochs = 50

In [25]:
opt = Adam(learning_rate= 2e-3)

In [26]:
vgg_based = VGG19(weights="imagenet", include_top=False, input_shape=input_shape)

80134624/80134624 [==============================] - 3s 0us/step


In [27]:
resnet_based = ResNet101(weights="imagenet", include_top=False, input_shape=input_shape)

171446536/171446536 [==============================] - 6s 0us/step


In [28]:
model_vgg = Sequential()

In [29]:
model_vgg.add(Input(shape= input_shape))
model_vgg.add(Rescaling(scale= 1./255))
model_vgg.add(vgg_based)
model_vgg.add(Flatten())
model_vgg.add(Dense(units= 1, activation= 'sigmoid'))

In [30]:
model_vgg.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 1)                 25089     
                                                                 
Total params: 20,049,473
Trainable params: 20,049,473
Non-trainable params: 0
_________________________________________________________________


In [31]:
model_vgg.compile(loss= 'binary_crossentropy', metrics= ['accuracy'], optimizer= opt)

In [32]:
model_vgg.fit(train, epochs= epochs)

Epoch 1/50
185/185 [==============================] - 102s 446ms/step - loss: 16.8526 - accuracy: 0.5245
Epoch 2/50
185/185 [==============================] - 86s 465ms/step - loss: 0.6995 - accuracy: 0.5326
Epoch 3/50
185/185 [==============================] - 87s 470ms/step - loss: 0.6917 - accuracy: 0.5392
Epoch 4/50
185/185 [==============================] - 87s 469ms/step - loss: 0.6966 - accuracy: 0.5353
Epoch 5/50
185/185 [==============================] - 87s 467ms/step - loss: 0.6875 - accuracy: 0.5590
Epoch 6/50
185/185 [==============================] - 86s 466ms/step - loss: 0.6855 - accuracy: 0.5564
Epoch 7/50
185/185 [==============================] - 86s 466ms/step - loss: 0.6854 - accuracy: 0.5625
Epoch 8/50
185/185 [==============================] - 85s 459ms/step - loss: 32.0769 - accuracy: 0.5272
Epoch 9/50
185/185 [==============================] - 82s 444ms/step - loss: 0.6928 - accuracy: 0.5421
Epoch 10/50
185/185 [==============================] - 83s 445ms/step 

In [33]:
model_vgg.evaluate(test)

61/61 [==============================] - 15s 171ms/step - loss: 0.6874 - accuracy: 0.5537


[0.6874120235443115, 0.5536635518074036]

In [34]:
model_resnet = Sequential()

In [35]:
model_resnet.add(Input(shape= input_shape))
model_resnet.add(Rescaling(scale= 1./255))
model_resnet.add(resnet_based)
model_resnet.add(Flatten())
model_resnet.add(Dense(units= 1, activation= 'sigmoid'))

In [36]:
model_resnet.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 resnet101 (Functional)      (None, 7, 7, 2048)        42658176  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_1 (Dense)             (None, 1)                 100353    
                                                                 
Total params: 42,758,529
Trainable params: 42,653,185
Non-trainable params: 105,344
_________________________________________________________________


In [39]:
model_resnet.compile(loss= 'binary_crossentropy', metrics= ['accuracy'], optimizer= 'adam')

In [40]:
model_resnet.fit(train, epochs= epochs)

Epoch 1/50
185/185 [==============================] - 166s 528ms/step - loss: 2.3377 - accuracy: 0.5353
Epoch 2/50
185/185 [==============================] - 101s 542ms/step - loss: 1.0882 - accuracy: 0.5387
Epoch 3/50
185/185 [==============================] - 100s 538ms/step - loss: 0.8117 - accuracy: 0.5524
Epoch 4/50
185/185 [==============================] - 100s 537ms/step - loss: 0.7677 - accuracy: 0.5529
Epoch 5/50
185/185 [==============================] - 99s 536ms/step - loss: 0.8171 - accuracy: 0.5498
Epoch 6/50
185/185 [==============================] - 100s 537ms/step - loss: 0.7569 - accuracy: 0.5579
Epoch 7/50
185/185 [==============================] - 100s 537ms/step - loss: 0.7390 - accuracy: 0.5703
Epoch 8/50
185/185 [==============================] - 102s 549ms/step - loss: 0.7030 - accuracy: 0.5986
Epoch 9/50
185/185 [==============================] - 102s 549ms/step - loss: 0.6960 - accuracy: 0.6157
Epoch 10/50
185/185 [==============================] - 101s 543ms

In [41]:
model_resnet.evaluate(test)

61/61 [==============================] - 16s 174ms/step - loss: 1.8643 - accuracy: 0.6362


[1.864321231842041, 0.6362228989601135]